<a href="https://colab.research.google.com/github/jeenathampi/ANNSearch/blob/master/ANNSearchAlgo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Approximate Nearest Neighbor Search

##Load all the necessary libraries

In [1]:
!pip install lightfm
from lightfm import LightFM
from lightfm.datasets import fetch_stackexchange
import pickle
!pip install faiss-cpu --no-cache
import faiss

!pip install annoy
import annoy

!pip install nmslib
import nmslib

     |████████████████████████████████| 310 kB 8.5 MB/s 
  Created wheel for lightfm: filename=lightfm-1.16-cp37-cp37m-linux_x86_64.whl size=705352 sha256=70a9e7f2a9bc3675c3284b3836e879a34a002392ffda63f4879dba4a3f8afbb6
  Stored in directory: /root/.cache/pip/wheels/f8/56/28/5772a3bd3413d65f03aa452190b00898b680b10028a1021914
Successfully built lightfm
     |████████████████████████████████| 8.4 MB 1.0 MB/s 
     |████████████████████████████████| 646 kB 6.8 MB/s 
  Created wheel for annoy: filename=annoy-1.17.0-cp37-cp37m-linux_x86_64.whl size=391681 sha256=1f69960f54a0e10eacaab3f9890cfad613c93134ce089df651a39016618bb6c5
  Stored in directory: /root/.cache/pip/wheels/4f/e8/1e/7cc9ebbfa87a3b9f8ba79408d4d31831d67eea918b679a4c07
Successfully built annoy
     |████████████████████████████████| 13.5 MB 76 kB/s 
     |████████████████████████████████| 188 kB 52.1 MB/s 


##Fetch Stackexchange dataset

In [2]:
stackexchange = fetch_stackexchange('crossvalidated')

In [3]:
train = stackexchange['train']
test = stackexchange['test']

model = LightFM(learning_rate=0.05, loss='warp', no_components=64, item_alpha=0.001)
model.fit_partial(train, item_features=stackexchange['item_features'], epochs=20 )

item_vectors = stackexchange['item_features'] * model.item_embeddings

In [4]:
with open('stackexchange.pickle', 'wb') as f:
    pickle.dump({"name": stackexchange['item_feature_labels'], "vector": item_vectors}, f)

In [5]:
def load_data():
    with open('stackexchange.pickle', 'rb') as f:
        data = pickle.load(f)
    return data

data = load_data()
data

{'name': array(['question_id:0', 'question_id:1', 'question_id:2', ...,
        'question_id:72357', 'question_id:72358', 'question_id:72359'],
       dtype='<U17'),
 'vector': array([[ 9.3440279e-02,  1.9732678e-02, -1.9864440e-02, ...,
          1.3154958e-01,  8.9234404e-02, -9.2519365e-02],
        [-3.9647970e-02, -2.5713187e-02, -9.0683931e-03, ...,
          2.9131718e-02,  6.0423020e-02, -1.3308075e-01],
        [ 5.1907957e-02,  9.5002148e-03,  7.6833814e-02, ...,
         -6.7052200e-02,  7.5678587e-02,  2.8238481e-02],
        ...,
        [ 8.9592505e-03,  4.9818680e-03,  3.1846061e-02, ...,
         -1.5749639e-02,  2.6674976e-03, -1.3177467e-02],
        [-9.4052507e-03,  3.4292003e-03,  2.6023377e-02, ...,
         -5.1021464e-03,  3.3302605e-02,  1.0804808e-02],
        [-2.0821518e-04, -1.1662702e-02,  1.5174535e-02, ...,
         -3.2871828e-04, -4.4604290e-05,  1.3499340e-02]], dtype=float32)}

##Exhaustive Search

###Create the index class

In [6]:
#Exhaustive Search
class ExactIndex():
    def __init__(self, vectors, labels):
        self.dimension = vectors.shape[1]
        self.vectors = vectors.astype('float32')
        self.labels = labels    
   
    def build(self):
        self.index = faiss.IndexFlatL2(self.dimension,)
        self.index.add(self.vectors)
        
    def query(self, vectors, k=10):
        distances, indices = self.index.search(vectors, k) 
        # I expect only query on one vector thus the slice
        return [self.labels[i] for i in indices[0]]

###Build the index

In [7]:
index = ExactIndex(data["vector"], data["name"])
index.build()

###Search for questions

Search for similar to question id 0. We can see below that we get approximate results.

In [8]:
index.query(data['vector'][0:1])

['question_id:0',
 'question_id:212',
 'question_id:225',
 'question_id:508',
 'question_id:3',
 'question_id:357',
 'question_id:2510',
 'question_id:239',
 'question_id:224',
 'question_id:4418']

##Locality Sensitive Hashing

In LSH, a hash table is constructed as their data structure by mapping points that are nearby into the same bucket.

###Create the index class

In [9]:
#LSH
class LSHIndex():
    def __init__(self, vectors, labels):
        self.dimension = vectors.shape[1]
        self.vectors = vectors.astype('float32')
        self.labels = labels    
   
    def build(self, num_bits=8):
        self.index = faiss.IndexLSH(self.dimension, num_bits)
        self.index.add(self.vectors)
        
    def query(self, vectors, k=10):
        distances, indices = self.index.search(vectors, k) 
        # I expect only query on one vector thus the slice
        return [self.labels[i] for i in indices[0]]

###Build the index

In [10]:
index = LSHIndex(data["vector"], data["name"])
index.build()

###Search

We get approximate results to search for questions similar to question id 0.

In [11]:
index.query(data['vector'][0:1])

['question_id:105',
 'question_id:431',
 'question_id:299',
 'question_id:12',
 'question_id:659',
 'question_id:876',
 'question_id:1300',
 'question_id:1415',
 'question_id:191',
 'question_id:0']

##Product Quantization

###Create the index class

Create the index class where we can control the subvector_size, number_of_partitions and search_in_x_partitions

In [12]:
#Product Quantization
class IVPQIndex():
    def __init__(self, vectors, labels):
        self.dimension = vectors.shape[1]
        self.vectors = vectors.astype('float32')
        self.labels = labels    
    def build(self, number_of_partition=8, search_in_x_partitions=2, subvector_size=8):
        quantizer = faiss.IndexFlatL2(self.dimension)
        self.index = faiss.IndexIVFPQ(quantizer, 
                                      self.dimension, 
                                      number_of_partition, 
                                      search_in_x_partitions, 
                                      subvector_size)
        self.index.train(self.vectors)
        self.index.add(self.vectors)
        
    def query(self, vectors, k=10):
        distances, indices = self.index.search(vectors, k) 
        # I expect only query on one vector thus the slice
        return [self.labels[i] for i in indices[0]]

###Build the index

In [13]:
index = IVPQIndex(data["vector"], data["name"])
index.build()

###Search

We get approximate results to search for questions similar to question id 0.

In [14]:
index.query(data['vector'][0:1])

['question_id:698',
 'question_id:363',
 'question_id:819',
 'question_id:0',
 'question_id:337',
 'question_id:319',
 'question_id:858',
 'question_id:1391',
 'question_id:257',
 'question_id:160']

##Trees and Graphs

###Create the index class

Create the index class where we can control the number_of_trees and search_k

In [15]:
#Trees
class AnnoyIndex():
    def __init__(self, vectors, labels):
        self.dimention = vectors.shape[1]
        self.vectors = vectors.astype('float32')
        self.labels = labels


    def build(self, number_of_trees=5):
        self.index = annoy.AnnoyIndex(self.dimention)
        for i, vec in enumerate(self.vectors):
            self.index.add_item(i, vec.tolist())
        self.index.build(number_of_trees)
        
    def query(self, vector, k=10):
        indices = self.index.get_nns_by_vector(vector.tolist(), k)
        return [self.labels[i] for i in indices]

###Build the index

In [16]:
index = AnnoyIndex(data["vector"], data["name"])
index.build()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: FutureWarning: The default argument for metric will be removed in future version of Annoy. Please pass metric='angular' explicitly.
  # Remove the CWD from sys.path while we load stuff.


###Search

We get approximate results to search for questions similar to question id 0.

In [17]:
index.query(data['vector'][0])

['question_id:0',
 'question_id:1712',
 'question_id:214',
 'question_id:360',
 'question_id:629',
 'question_id:1107',
 'question_id:1244',
 'question_id:1478',
 'question_id:264',
 'question_id:10594']

##HSNW

###Create the index class

In [18]:
#hsnw
class NMSLIBIndex():
    def __init__(self, vectors, labels):
        self.dimention = vectors.shape[1]
        self.vectors = vectors.astype('float32')
        self.labels = labels
    def build(self):
        self.index = nmslib.init(method='hnsw', space='cosinesimil')
        self.index.addDataPointBatch(self.vectors)
        self.index.createIndex({'post': 2})
        
    def query(self, vector, k=10):
        indices = self.index.knnQuery(vector, k=k)
        return [self.labels[i] for i in indices[0]]

###Build index

In [19]:
index = NMSLIBIndex(data["vector"], data["name"])
index.build()

###Search

We get approximate results to search for questions similar to question id 0.

In [20]:
index.query(data['vector'][0])

['question_id:0',
 'question_id:212',
 'question_id:3',
 'question_id:225',
 'question_id:508',
 'question_id:90',
 'question_id:357',
 'question_id:239',
 'question_id:257',
 'question_id:46']